In [11]:
import pandas as pd
import scipy.stats

In [2]:
bike_path = "clean/bike_data.csv"
city_path = "clean/Cities.csv"
bike_scores = pd.read_csv(bike_path)
weather_scores = pd.read_csv(city_path)
bike_scores

,City,Overall Score,Ridership,Safety,Acceleration,Network,Reach
0,"SPRINGDALE, AR",3.0,1.3,2.2,3.0,2.2,4.0
1,"ENID, OK",2.4,1.0,1.6,NaN,3.2,3.8
2,"DUBUQUE, IA",2.1,1.6,2.3,0.5,2.2,3.8
3,"SOUTH BEND, IN",2.0,1.7,1.9,0.6,2.1,3.7
4,"CRESTED BUTTE, CO",2.6,3.5,2.8,0.7,2.5,3.6
...,...,...,...,...,...,...,...
505,"BRIELLE, NJ",0.6,1.0,0.4,NaN,0.8,0.0
506,"TATAMY, PA",0.5,0.9,0.2,NaN,0.8,0.0
507,"FOXBOROUGH, MA",0.5,0.9,0.2,NaN,0.8,0.0
508,"HOOKSETT, NH",0.5,0.8,0.2,NaN,0.8,0.0


In [3]:
weather_scores = weather_scores.drop('Unnamed: 0', axis = 1)
weather_scores['City'] = weather_scores['City'].str.upper()
weather_scores

,City,Value
0,"ARDEN-ARCADE, CA",1
1,"CITRUS HEIGHTS, CA",1
2,"ROSEVILLE, CA",1
3,"DAVIS, CA",1
4,"GLENDALE, CA",1
...,...,...
595,"SYRACUSE, NY",100
596,"YOUNGSTOWN, OH",100
597,"KETCHIKAN, AK",100
598,"SITKA, AK",100


In [4]:
#one csv had 510 other had 600 left with 317
merge_data = pd.merge(bike_scores, weather_scores, on = 'City')
merge_data

,City,Overall Score,Ridership,Safety,Acceleration,Network,Reach,Value
0,"SPRINGDALE, AR",3.0,1.3,2.2,3.0,2.2,4.0,55
1,"SOUTH BEND, IN",2.0,1.7,1.9,0.6,2.1,3.7,86
2,"KALAMAZOO, MI",1.9,1.7,1.7,0.6,2.2,3.6,85
3,"SPRINGFIELD, MO",1.9,1.8,1.3,0.5,2.2,3.5,63
4,"MEDFORD, OR",1.9,1.1,1.8,NaN,1.6,3.2,39
...,...,...,...,...,...,...,...,...
312,"ROCKFORD, IL",1.3,1.6,1.4,0.5,2.1,0.9,96
313,"CLARKSVILLE, TN",1.1,1.3,1.1,0.3,2.0,0.8,53
314,"CHEYENNE, WY",1.2,1.6,1.3,0.4,2.1,0.7,61
315,"WHEELING, WV",1.2,0.8,1.8,NaN,1.6,0.5,82


In [23]:
merge_data.to_csv('clean/merged_data.csv', index = False, header = True)

In [14]:
merge_data[['City Name','State']] = merge_data.City.str.split(", ",expand=True) 
#merge_data.to_csv('clean/merege2_data.csv', index = False, header = True)
pregroup = merge_data.drop('City', axis = 1)
pregroup.to_csv('clean/merge2_data.csv', index = False, header = True)
pregroup.rename(columns={"Overall Score": "Bike Score", "Value": "Good Weather %ile", "City Name" : "City"})
pregroup.head()

,Overall Score,Ridership,Safety,Acceleration,Network,Reach,Value,City Name,State
0,3.0,1.3,2.2,3.0,2.2,4.0,55,SPRINGDALE,AR
1,2.0,1.7,1.9,0.6,2.1,3.7,86,SOUTH BEND,IN
2,1.9,1.7,1.7,0.6,2.2,3.6,85,KALAMAZOO,MI
3,1.9,1.8,1.3,0.5,2.2,3.5,63,SPRINGFIELD,MO
4,1.9,1.1,1.8,NaN,1.6,3.2,39,MEDFORD,OR


In [12]:
pregroup.groupby(['State']).mean()

KeyError: 'Stats'